In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
import plotly.express as px
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [3]:
df = pd.read_csv('dataframes/df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [4]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_processor_ordinal(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_ordinal_encoder(trial)

    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

# Lasso

In [7]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [8]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=200)

[I 2024-07-25 10:58:23,794] A new study created in memory with name: optimization
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.753e-02, tolerance: 2.415e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.144e-02, tolerance: 2.409e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of th

[I 2024-07-25 10:58:31,975] Trial 7 finished with value: 0.9407264490840564 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 6.478886532606612e-05}. Best is trial 5 with value: 0.9504529057304483.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.093e-02, tolerance: 2.415e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.542e-02, tolerance: 2.409e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

[I 2024-07-25 10:58:59,168] Trial 26 finished with value: 0.9417441538370372 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'ordinal', 'alpha': 0.0002350651207763719}. Best is trial 25 with value: 0.9504619036661117.
[I 2024-07-25 10:58:59,817] Trial 27 finished with value: 0.946902621321246 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006324653594778544}. Best is trial 25 with value: 0.9504619036661117.
[I 2024-07-25 10:59:01,293] Trial 28 finished with value: 0.9504825674291144 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00013964514205982436}. Best is trial 28 with value: 0.9504825674291144.
[I 2024-07-25 10:59:02,207] Trial 29 finished with value: 0.9416691062939966 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'ordinal', 'alpha': 7.543026208387515e-05}. Best is trial 28 with value: 0.9504825674291144.
[I 2024-07-25 10:59:03,458] Trial 30 f

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.254e+00, tolerance: 2.345e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.568e+00, tolerance: 2.415e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.468e+00, toleranc

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-07-25 10:59:45,090] Trial 77 finished with value: 0.9500167828520556 and para

[I 2024-07-25 11:00:02,936] Trial 95 finished with value: 0.9507277958188898 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003971020815943377}. Best is trial 88 with value: 0.9508124811471179.
[I 2024-07-25 11:00:03,470] Trial 96 finished with value: 0.9417353760229847 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.00028628695794383826}. Best is trial 88 with value: 0.9508124811471179.
[I 2024-07-25 11:00:04,355] Trial 97 finished with value: 0.9505583345252102 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00021578751688326747}. Best is trial 88 with value: 0.9508124811471179.
[I 2024-07-25 11:00:05,040] Trial 98 finished with value: 0.9505238791019778 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0005003828045530127}. Best is trial 88 with value: 0.9508124811471179.
[I 2024-07-25 11:00:05,

[I 2024-07-25 11:00:26,087] Trial 122 finished with value: 0.9508116478512599 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00031051120233442333}. Best is trial 104 with value: 0.9508124821158186.
[I 2024-07-25 11:00:26,989] Trial 123 finished with value: 0.9506930330947224 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00024650116552103404}. Best is trial 104 with value: 0.9508124821158186.
[I 2024-07-25 11:00:27,725] Trial 124 finished with value: 0.9508118086653601 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00030978212796107125}. Best is trial 104 with value: 0.9508124821158186.
[I 2024-07-25 11:00:28,489] Trial 125 finished with value: 0.9508113216414141 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003169971490382569}. Best is trial 104 with value: 0.9508124821158186.
[

[I 2024-07-25 11:00:49,360] Trial 151 finished with value: 0.9508094956354782 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.000294437865126824}. Best is trial 104 with value: 0.9508124821158186.
[I 2024-07-25 11:00:50,101] Trial 152 finished with value: 0.9508125679921836 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003059499593517016}. Best is trial 152 with value: 0.9508125679921836.
[I 2024-07-25 11:00:50,784] Trial 153 finished with value: 0.950802212048092 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00034134307114919583}. Best is trial 152 with value: 0.9508125679921836.
[I 2024-07-25 11:00:51,553] Trial 154 finished with value: 0.950800035812323 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00028388736953979544}. Best is trial 152 with value: 0.9508125679921836.
[I 20

[I 2024-07-25 11:01:11,820] Trial 180 finished with value: 0.9507397148337453 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003888149967628445}. Best is trial 152 with value: 0.9508125679921836.
[I 2024-07-25 11:01:12,593] Trial 181 finished with value: 0.950812427632443 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003049211290600782}. Best is trial 152 with value: 0.9508125679921836.
[I 2024-07-25 11:01:13,457] Trial 182 finished with value: 0.950790976344598 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00027663264767635896}. Best is trial 152 with value: 0.9508125679921836.
[I 2024-07-25 11:01:14,251] Trial 183 finished with value: 0.9508115075162349 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003114330508442623}. Best is trial 152 with value: 0.9508125679921836.
[I 20

In [9]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.9508125679921836
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0003059499593517016}


In [10]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

,Trial,Score,Parameters
0,0,0.941096,"[(scaling_method, minmax), (encoding_method, o..."
1,1,0.941321,"[(scaling_method, standard), (encoding_method,..."
2,2,0.947896,"[(scaling_method, robust), (encoding_method, o..."
3,3,0.948237,"[(scaling_method, minmax), (encoding_method, o..."
4,4,0.941095,"[(scaling_method, maxabs), (encoding_method, o..."
...,...,...,...
195,195,0.950751,"[(scaling_method, robust), (encoding_method, o..."
196,196,0.950811,"[(scaling_method, robust), (encoding_method, o..."
197,197,0.950773,"[(scaling_method, robust), (encoding_method, o..."
198,198,0.950804,"[(scaling_method, robust), (encoding_method, o..."


In [15]:
pd.set_option('display.max_colwidth', None) 
optuna_lasso.Parameters

0                   [(scaling_method, minmax), (encoding_method, ordinal), (alpha, 1.6999779739443475e-05)]
1                 [(scaling_method, standard), (encoding_method, ordinal), (alpha, 0.00037901740871573876)]
2      [(scaling_method, robust), (encoding_method, onehot), (drop, first), (alpha, 0.0009834045353742098)]
3      [(scaling_method, minmax), (encoding_method, onehot), (drop, None), (alpha, 2.5666965172816898e-05)]
4                   [(scaling_method, maxabs), (encoding_method, ordinal), (alpha, 1.0571302084059589e-05)]
                                                       ...                                                 
195     [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.0003814762162935689)]
196    [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.00031881502123266083)]
197     [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.0002681604864862338)]
198    [(scaling_method, rob

In [18]:
 optuna_lasso['alpha'] = optuna_lasso.Parameters.apply(lambda x: x[-1][1])

In [23]:
 optuna_lasso['encoding'] = optuna_lasso.Parameters.apply(lambda x: x[1][1])

In [24]:
optuna_lasso['scaling'] = optuna_lasso.Parameters.apply(lambda x: x[0][1])

In [25]:
optuna_lasso.to_csv('dataframes/optuna/optuna_lasso.csv')

In [27]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'encoding',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# fig.write_image("images/optuna_lasso.png")

In [ ]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'alpha',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# Ridge

In [41]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [43]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

#study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=20)

[I 2024-07-11 09:50:20,011] A new study created in memory with name: optimization


In [17]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9496764273611051
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'alpha': 24.817075515507273}


# EN

In [39]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [40]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

#study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:19,990] A new study created in memory with name: optimization


In [66]:
# after adding in the L1 value
print(study_en.best_value)
print(study_en.best_params)

0.9508214083737423
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006921322276640134, 'l1_ratio': 0.4415555448483296}


In [29]:
print(study_en.best_value)
print(study_en.best_params)

0.950820633330468
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006178621404820522}


# XGB

In [13]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", .001, .1, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 4),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5)
    }

    return XGBRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_xgb(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_xgb(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [8]:
from optuna import create_study

study_xgb = create_study(study_name='optimization', direction='maximize')

study_xgb.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=200)

[I 2024-07-16 09:19:54,516] A new study created in memory with name: optimization
[I 2024-07-16 09:19:56,514] Trial 0 finished with value: 0.9346673308494285 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 475, 'learning_rate': 0.024978041209914686, 'max_depth': 2, 'subsample': 0.3669553381298013, 'colsample_bytree': 0.8770501936980049, 'min_child_weight': 2}. Best is trial 0 with value: 0.9346673308494285.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encode

[I 2024-07-16 09:20:33,152] Trial 13 finished with value: 0.9499376286957801 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 714, 'learning_rate': 0.0397794641187814, 'max_depth': 4, 'subsample': 0.37871350729210107, 'colsample_bytree': 0.7245464695981462, 'min_child_weight': 4}. Best is trial 12 with value: 0.9521593794302905.
[I 2024-07-16 09:20:36,416] Trial 14 finished with value: 0.9292365942483789 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'n_estimators': 677, 'learning_rate': 0.010491864708705173, 'max_depth': 4, 'subsample': 0.6213820095830929, 'colsample_bytree': 0.7574052781594777, 'min_child_weight': 3}. Best is trial 12 with value: 0.9521593794302905.
[I 2024-07-16 09:20:42,213] Trial 15 finished with value: 0.9511581767669357 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 798, 'learning_rate': 0.047391052672483856, 'max_dept

[I 2024-07-16 09:21:44,838] Trial 29 finished with value: 0.949164104945081 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 920, 'learning_rate': 0.025607394808080226, 'max_depth': 3, 'subsample': 0.30284178066813316, 'colsample_bytree': 0.7437036929866911, 'min_child_weight': 1}. Best is trial 12 with value: 0.9521593794302905.
[I 2024-07-16 09:21:47,979] Trial 30 finished with value: 0.8579720996612158 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 487, 'learning_rate': 0.010463185713603128, 'max_depth': 4, 'subsample': 0.6428368830901725, 'colsample_bytree': 0.794344040387567, 'min_child_weight': 3}. Best is trial 12 with value: 0.9521593794302905.
[I 2024-07-16 09:21:52,466] Trial 31 finished with value: 0.9507158059346441 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 793, 'learning_rate': 0.06662733767657203,

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-07-16 09:22:58,527] Trial 47 finished with value: 0.950813334055247 and param

[I 2024-07-16 09:24:11,718] Trial 63 finished with value: 0.950095679462763 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 845, 'learning_rate': 0.06669732972994195, 'max_depth': 4, 'subsample': 0.308519083787203, 'colsample_bytree': 0.7721243771984889, 'min_child_weight': 4}. Best is trial 52 with value: 0.9528428538331914.
[I 2024-07-16 09:24:16,603] Trial 64 finished with value: 0.951190265804146 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 809, 'learning_rate': 0.05185357281894903, 'max_depth': 4, 'subsample': 0.36306012279158645, 'colsample_bytree': 0.752193401428429, 'min_child_weight': 4}. Best is trial 52 with value: 0.9528428538331914.
[I 2024-07-16 09:24:20,537] Trial 65 finished with value: 0.9503587055162651 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 722, 'learning_rate': 0.04249423378391319, 'max_

...

[I 2024-07-16 09:28:55,783] Trial 120 finished with value: 0.9516734162647875 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 844, 'learning_rate': 0.039017143883343756, 'max_depth': 4, 'subsample': 0.6711227617771436, 'colsample_bytree': 0.5441787248768, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:29:00,469] Trial 121 finished with value: 0.9520272425511326 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 813, 'learning_rate': 0.051019276013019456, 'max_depth': 4, 'subsample': 0.5864869288457577, 'colsample_bytree': 0.5256280564115579, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:29:06,267] Trial 122 finished with value: 0.9514141067693501 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 876, 'learning_rate': 0.04537371525589987, 'm

[I 2024-07-16 09:30:31,587] Trial 139 finished with value: 0.9514623040757304 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 944, 'learning_rate': 0.045301077531780344, 'max_depth': 4, 'subsample': 0.5918056218415126, 'colsample_bytree': 0.5482131726886658, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:30:36,914] Trial 140 finished with value: 0.9515829560380045 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 911, 'learning_rate': 0.026725828511096044, 'max_depth': 4, 'subsample': 0.5359489610458162, 'colsample_bytree': 0.5582612046747151, 'min_child_weight': 2}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:30:42,469] Trial 141 finished with value: 0.9515024391257885 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 983, 'learning_rate': 0.044005657977

[I 2024-07-16 09:32:12,724] Trial 158 finished with value: 0.9440032990477017 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 962, 'learning_rate': 0.01288321650876376, 'max_depth': 4, 'subsample': 0.6648151784081523, 'colsample_bytree': 0.5193159596676827, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:32:17,600] Trial 159 finished with value: 0.9498207934424855 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'ordinal', 'n_estimators': 999, 'learning_rate': 0.03379167404799594, 'max_depth': 4, 'subsample': 0.6370986110616893, 'colsample_bytree': 0.5000280260413144, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.
[I 2024-07-16 09:32:23,457] Trial 160 finished with value: 0.9517190306038683 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 931, 'learning_rate': 0.039906270062291686, 'max_dept

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [3, 9, 14] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [9, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227

[I 2024-07-16 09:35:50,691] Trial 199 finished with value: 0.9516026708929515 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 903, 'learning_rate': 0.039354403596329995, 'max_depth': 4, 'subsample': 0.6250221191525293, 'colsample_bytree': 0.5290022910090753, 'min_child_weight': 3}. Best is trial 76 with value: 0.9538790152827454.


In [9]:
print(study_xgb.best_value)
print(study_xgb.best_params)

0.9538790152827454
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 905, 'learning_rate': 0.04643973848790804, 'max_depth': 4, 'subsample': 0.5797329944770706, 'colsample_bytree': 0.527504939004116, 'min_child_weight': 3}


In [10]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_xgb.get_trials()[trial].number)
    score.append(study_xgb.get_trials()[trial].value)
    parameters.append(list(study_xgb.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_xgb = pd.DataFrame.from_dict(optuna_dict)
optuna_xgb

,Trial,Score,Parameters
0,0,0.934667,"[(scaling_method, robust), (encoding_method, o..."
1,1,-6.163066,"[(scaling_method, standard), (encoding_method,..."
2,2,0.946490,"[(scaling_method, robust), (encoding_method, o..."
3,3,-6.347293,"[(scaling_method, robust), (encoding_method, o..."
4,4,0.949246,"[(scaling_method, maxabs), (encoding_method, o..."
...,...,...,...
195,195,0.952707,"[(scaling_method, standard), (encoding_method,..."
196,196,0.951697,"[(scaling_method, standard), (encoding_method,..."
197,197,0.951567,"[(scaling_method, standard), (encoding_method,..."
198,198,0.951890,"[(scaling_method, standard), (encoding_method,..."


In [ ]:
[(scaling_method, standard), 
 (encoding_method, ordinal),
 (n_estimators, 917), 
 (learning_rate, 0.044081018318051106), 
 (max_depth, 4), (subsample, 0.6114930290253439), 
 (colsample_bytree, 0.5330333973787846), (min_child_weight, 2)]


In [44]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)
optuna_xgb.sort_values(by='Score', ascending=False)

,Trial,Score,Parameters
76,76,0.953879,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 905), (learning_rate, 0.04643973848790804), (max_depth, 4), (subsample, 0.5797329944770706), (colsample_bytree, 0.527504939004116), (min_child_weight, 3)]"
154,154,0.953849,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 981), (learning_rate, 0.047198601971795234), (max_depth, 4), (subsample, 0.5955423388155735), (colsample_bytree, 0.5091959392090488), (min_child_weight, 3)]"
161,161,0.953627,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 974), (learning_rate, 0.04740516101352134), (max_depth, 4), (subsample, 0.5914713646169084), (colsample_bytree, 0.5103526150143552), (min_child_weight, 3)]"
155,155,0.953577,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 978), (learning_rate, 0.04611315421181104), (max_depth, 4), (subsample, 0.6032259102409443), (colsample_bytree, 0.5097594989102657), (min_child_weight, 3)]"
142,142,0.953422,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 967), (learning_rate, 0.037077274389072025), (max_depth, 4), (subsample, 0.6052407177992929), (colsample_bytree, 0.509636780573371), (min_child_weight, 3)]"
85,85,0.953407,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 975), (learning_rate, 0.042596843254780296), (max_depth, 4), (subsample, 0.5904568978593641), (colsample_bytree, 0.5156085878989596), (min_child_weight, 2)]"
181,181,0.953403,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 982), (learning_rate, 0.042994378010402366), (max_depth, 4), (subsample, 0.568554119025959), (colsample_bytree, 0.5136219028880783), (min_child_weight, 3)]"
133,133,0.953348,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 951), (learning_rate, 0.04307704330028825), (max_depth, 4), (subsample, 0.5696307183664032), (colsample_bytree, 0.5380070898897199), (min_child_weight, 3)]"
81,81,0.953341,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 937), (learning_rate, 0.04021124807781962), (max_depth, 4), (subsample, 0.5555256528827893), (colsample_bytree, 0.5462831730811686), (min_child_weight, 3)]"
152,152,0.953248,"[(scaling_method, standard), (encoding_method, onehot), (drop, None), (n_estimators, 971), (learning_rate, 0.0458436225743243), (max_depth, 4), (subsample, 0.5898992546162253), (colsample_bytree, 0.5152225243663119), (min_child_weight, 3)]"


In [59]:
optuna_xgb['encoding'] = optuna_xgb.Parameters.apply(lambda x: x[1][1])

In [65]:
fig = px.scatter(optuna_xgb,
                 x="Trial", 
                 y="Score", 
                 color = 'encoding',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9539</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# RF

In [37]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_samples_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [38]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

#study_rf.optimize(lambda trial: objective_rf(trial, X, y), n_trials=5)

[I 2024-07-11 09:43:39,468] A new study created in memory with name: optimization
[I 2024-07-11 09:44:37,413] Trial 0 finished with value: 0.9083545094739573 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 253, 'max_depth': 60, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:45:24,724] Trial 1 finished with value: 0.8999706968816368 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 328, 'max_depth': 68, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:46:51,911] Trial 2 finished with value: 0.9081071301156255 and parameters: {'scaling_method': 'minmax', 'bootstrap': True, 'n_estimators': 427, 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:48:22,636] Trial 3 finished with value: 0.9093564157184439 and parameters: {'sc

In [51]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9106961840498735
{'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 338, 'max_depth': 73, 'min_sample_split': 4, 'min_sample_leaf': 2}


# gbr

In [31]:
def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_gbr(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_gbr(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [34]:
from optuna import create_study

study_gbr = create_study(study_name='optimization', direction='maximize')

study_gbr.optimize(lambda trial: objective_gbr(trial, X, y), n_trials=5)

[I 2024-07-11 09:36:55,471] A new study created in memory with name: optimization
[I 2024-07-11 09:38:00,789] Trial 0 finished with value: 0.10676969273467858 and parameters: {'scaling_method': 'maxabs', 'n_estimators': 680, 'learning_rate': 0.00015291122193602163, 'max_depth': 6, 'subsample': 0.7453028457760895, 'min_sample_split': 10, 'min_sample_leaf': 9}. Best is trial 0 with value: 0.10676969273467858.
[I 2024-07-11 09:38:22,650] Trial 1 finished with value: 0.28100365000054095 and parameters: {'scaling_method': 'standard', 'n_estimators': 605, 'learning_rate': 0.0006146473911796216, 'max_depth': 2, 'subsample': 0.8742574967407413, 'min_sample_split': 8, 'min_sample_leaf': 2}. Best is trial 1 with value: 0.28100365000054095.
[I 2024-07-11 09:39:24,934] Trial 2 finished with value: 0.2466584750875623 and parameters: {'scaling_method': 'standard', 'n_estimators': 533, 'learning_rate': 0.00043180685670930226, 'max_depth': 6, 'subsample': 0.8638913349214837, 'min_sample_split': 8, 'mi

In [35]:
print(study_gbr.best_value)
print(study_gbr.best_params)

0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_leaf': 10}


# Adaboost

In [53]:
def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ada(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ada(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [54]:
from optuna import create_study

study_ada = create_study(study_name='optimization', direction='maximize')

study_ada.optimize(lambda trial: objective_ada(trial, X, y), n_trials=50)

[I 2024-07-11 10:02:52,147] A new study created in memory with name: optimization
[I 2024-07-11 10:02:52,518] Trial 0 finished with value: 0.7697034560200977 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5591223026880627, 'loss': 'linear', 'n_estimators': 5}. Best is trial 0 with value: 0.7697034560200977.
[I 2024-07-11 10:03:10,819] Trial 1 finished with value: 0.8546529541652251 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.06846055794485856, 'loss': 'linear', 'n_estimators': 404}. Best is trial 1 with value: 0.8546529541652251.
[I 2024-07-11 10:03:29,877] Trial 2 finished with value: 0.875164826045092 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5252293433374021, 'loss': 'exponential', 'n_estimators': 551}. Best is trial 2 with value: 0.875164826045092.
[I 2024-07-11 10:03:51,103] Trial 3 finished with value: 0.7894060614839619 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.006602759458806035, 'loss': 'square'

[I 2024-07-11 10:14:44,856] Trial 33 finished with value: 0.866576708984014 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.09929974514686613, 'loss': 'exponential', 'n_estimators': 581}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:14:58,969] Trial 34 finished with value: 0.8703748675462952 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.2245325690083747, 'loss': 'exponential', 'n_estimators': 368}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:17,983] Trial 35 finished with value: 0.876282939613254 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.5987616594370728, 'loss': 'exponential', 'n_estimators': 614}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:40,207] Trial 36 finished with value: 0.8102956663214451 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.013357794606257661, 'loss': 'exponential', 'n_estimators': 454}. Best is trial 10 with value: 0.878

# Loop through?

In [61]:
studies = [study_lasso, study_ridge, study_en, study_xgb, study_rf, study_gbr, study_ada]
objectives = [objective_lasso, 
              objective_ridge, 
              objective_en,
              objective_xgb,
              objective_rf,
              objective_gbr,
              objective_ada]

# for study, objective in zip(studies, objectives):
#     study.optimize(lambda trial: objective(trial, X, y), n_trials=2)
    

[<function __main__.objective_lasso(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_ridge(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_en(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_xgb(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: O

In [52]:
for study in studies:
    print (study.best_value)
    print (study.best_params)

0.8735606638813536
{'scaling_method': 'robust', 'learning_rate': 0.30065504792403025, 'loss': 'exponential', 'n_estimators': 428}
0.7980960053211354
{'scaling_method': 'robust', 'learning_rate': 0.006736329679908936, 'loss': 'exponential', 'n_estimators': 615}
0.8177296198440576
{'scaling_method': 'maxabs', 'learning_rate': 0.04083092875953665, 'loss': 'exponential', 'n_estimators': 196}
0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}
0.9094102521730991
{'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 494, 'max_depth': 62, 'min_samples_split': 4, 'min_samples_leaf': 1}
0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_l